<a href="https://colab.research.google.com/github/lustea0201/Interpretability/blob/master/HDattempt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
torch.manual_seed(0)
dtype = torch.float
from google.colab import drive
drive.mount('/content/drive/')
import zipfile
import sys
sys.path.append('local_modules')
!wget https://raw.githubusercontent.com/lustea0201/Interpretability/master/utils.py -P local_modules -nc
import local_modules.utils as utils

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
File ‘local_modules/utils.py’ already there; not retrieving.



In [0]:
import numpy as np
from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize
from torchvision.datasets import ImageFolder
import os
from torch.utils.data import DataLoader

net = utils.Net() # Model instantiation
# Loading the trained dictionnary state
net.load_state_dict(torch.load('drive/My Drive/Interpretability/model2.pth')) 

data = zipfile.ZipFile("/content/drive/My Drive/Interpretability/HD/data2/data.zip", 'r')

root_dir = "main_dir"
data.extractall(root_dir)
data.close()

Batch_Size = 200

MEAN, STD = np.array([0.5, 0.5, 0.5]), np.array([0.5, 0.5, 0.5])

transf = transforms.Compose( [ToTensor(), Normalize(mean=MEAN, std=STD)])

train_data = ImageFolder(root = os.path.join(root_dir, 'train'), transform = transf)
dataloader = DataLoader(train_data, batch_size = Batch_Size, shuffle = True, num_workers = 0)
train_loader = iter(dataloader)

In [132]:
X,y = next(train_loader)
correct = 0
for k in range(200):
  image = X[k]
  label = y[k].numpy()

  label_l = 0
  class FoundLabel1(Exception): pass

  try:
    for i in range(19): 
      for j in range(19): 
        s_image = image[:,i*50:i*50+100, j*60:j*60+120]
        input_ = s_image.view(-1, 3, 100, 120) # This shape is necessary for the network 
        output = net(input_)
        _, predicted = torch.max(output.data, 1)
        if (predicted == 1):
          raise FoundLabel1

  except FoundLabel1:
    label_l = 1

  if label_l == label: 
    correct += 1
print("success rate: %f " %(correct/len(y)*100)) 

success rate: 100.000000 
